<a href="https://colab.research.google.com/github/ShamankaA/math_prog/blob/main/6.%20%D0%90%D0%B2%D1%82%D0%BE%D1%8D%D0%BD%D0%BA%D0%BE%D0%B4%D0%B5%D1%80%D1%8B/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%966_%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B0%D0%B2%D1%82%D0%BE%D1%8D%D0%BD%D0%BA%D0%BE%D0%B4%D0%B5%D1%80%D0%BE%D0%B2_%D0%B4%D0%BB%D1%8F_%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BF%D1%80%D0%B8%D0%BA%D0%BB%D0%B0%D0%B4%D0%BD%D1%8B%D1%85_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Практическая работа №6. Проектирование автоэнкодеров для решения прикладных задач**

[**Ссылка на код с пары**](https://colab.research.google.com/drive/1KfCEQly5k0PUdCR1w-my2-QmYfFexdvK?usp=sharing)

### - **N.B.: Во всех заданиях, графический интерфейс для взаимодействия с обученной моделью, реализуется с помощью Gradio!**

### - **Для каждого задания в графическом интерфейсе должно быть встроено минимум 3 примера (sample)**

# Задание №1. Реализуйте автоэнкодер для колоризации чёрно-белых изображений

* **Чем больше объектов разных классов будет в исходном датасете, тем универсальнее будет работа обученной Вами модели**

> Например, если в Вашем датасете только кошки, то все объекты кроме кошек будут колоризоваться некорректно. Соответственно следует очень тщательно подойти к выбору датасета.

  * Для решения данной задачи может подойти датасет [CIFAR-100](https://www.kaggle.com/datasets/fedesoriano/cifar100), но его минус заключается в том, что разрешение изображений довольно низкое

* Хорошим вариантом будет использование датасета [ImageNet](https://paperswithcode.com/dataset/imagenet), но ввиду его объёмности, процесс обучения займет большое количество времени, поэтому Вы можете использовать одну из его [сокращенных версий](https://www.kaggle.com/datasets/ifigotin/imagenetmini-1000)

 **Конечный выбор датасета осуществляется по Вашему желанию, учитывая рекомендации приведенные выше.**


In [13]:
# Импорт библиотек
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import gradio as gr

In [14]:
# Загрузка датасета CIFAR-100
(x_train, _), (x_test, _) = tf.keras.datasets.cifar100.load_data()

In [15]:
# Нормализация и преобразование в ч/б
x_train_gray = tf.image.rgb_to_grayscale(x_train / 255.0)
x_test_gray = tf.image.rgb_to_grayscale(x_test / 255.0)
x_train_color = x_train / 255.0
x_test_color = x_test / 255.0


In [16]:
# Архитектура автоэнкодера
input_img = layers.Input(shape=(32, 32, 1))

In [17]:
# Энкодер
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

In [18]:
# Декодер
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [19]:
# Обучение
history = autoencoder.fit(
    x_train_gray, x_train_color,
    epochs=20,
    batch_size=128,
    validation_data=(x_test_gray, x_test_color)
)


Epoch 1/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 13s 26ms/step - loss: 0.0256 - val_loss: 0.0129
Epoch 2/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - loss: 0.0122 - val_loss: 0.0118
Epoch 3/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.0115 - val_loss: 0.0112
Epoch 4/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.0112 - val_loss: 0.0111
Epoch 5/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0107 - val_loss: 0.0105
Epoch 6/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.0105 - val_loss: 0.0106
Epoch 7/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.0103 - val_loss: 0.0101
Epoch 8/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.0101 - val_loss: 0.0102
Epoch 9/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0100 - val_loss: 0.0099
Epoch 10/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.0098 - val_loss: 0.0106
Epoch 11/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0098 - val_loss: 0.0099
Epoch 12/20
391/391 ━━━━━━━━━━━━━

In [28]:
# Функция для колоризации
def colorize(image):
    # Конвертация входного изображения в [0, 1]
    image = (image.astype('float32') / 255.0).reshape(1, 32, 32, 1)
    prediction = autoencoder.predict(image)
    return (prediction[0] * 255).astype('uint8')

In [29]:
# Подготовка примеров из тестового набора данных
samples = [(x_test_gray[i].numpy().squeeze() * 255).astype(np.uint8) for i in range(3)]

In [30]:
# Интерфейс Gradio
interface = gr.Interface(
    fn=colorize,
    inputs=gr.Image(height=256, width=256, image_mode='L'),  # Увеличиваем размеры ввода
    outputs=gr.Image(height=256, width=256),  # Увеличиваем размеры вывода
    examples=samples,
    title="Colorization Autoencoder",
    description="Загрузите черно-белое изображение для его колоризации."
)

interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a1e09c1d4b5507657c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Задание №2. Реализуйте автоэнкодер для удаления шума на однотипных изображениях

1.  Подберите датасет, состоящий из однотипных изображений, которые в реальной жизни часто подвержены зашумлению, например спутниковые снимки, ночные фотографии и т.д.

2.  Затем примените к ним операцию зашумления и обучите модель. Также учитывайте тип шума, который Вы применяете. Он должен быть приближен к естественному.

Хороший пример реализации подобной задачи: https://www.kaggle.com/code/michalbrezk/denoise-images-using-autoencoders-tf-keras

In [35]:
# Импорт библиотек
import tensorflow as tf
from tensorflow.keras import layers, Model
import gradio as gr
import numpy as np

# Загрузка датасета Fashion MNIST
(x_train, _), (x_test, _) = tf.keras.datasets.fashion_mnist.load_data()

# Нормализация и добавление шума
def add_noise(img, noise_factor=0.3):
    noisy = img + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=img.shape)
    return np.clip(noisy, 0., 1.)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train_noisy = add_noise(x_train)
x_test_noisy = add_noise(x_test)

# Добавление канального измерения
x_train_noisy = np.expand_dims(x_train_noisy, axis=-1)
x_test_noisy = np.expand_dims(x_test_noisy, axis=-1)

# Архитектура автоэнкодера
input_img = layers.Input(shape=(28, 28, 1))

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Обучение
autoencoder.fit(
    x_train_noisy, np.expand_dims(x_train, axis=-1),
    epochs=15,
    batch_size=128,
    validation_data=(x_test_noisy, np.expand_dims(x_test, axis=-1))
)

# Функция для удаления шума
def denoise(image):
    # Конвертация в градации серого, если изображение цветное
    if image.ndim == 3 and image.shape[2] == 3:
        image = np.dot(image[..., :3], [0.2989, 0.5870, 0.1140]).astype('float32')
    # Добавление канального измерения для черно-белых изображений
    if image.ndim == 2:
        image = np.expand_dims(image, axis=-1)
    # Изменение размера и нормализация
    image = tf.image.resize(image, (28, 28)).numpy()
    image = image.astype('float32') / 255.0
    # Предсказание
    prediction = autoencoder.predict(np.expand_dims(image, axis=0))
    # Преобразование в формат изображения
    output = (prediction[0] * 255).astype('uint8').squeeze()
    return output

# Подготовка примеров (нормализация к 0-255)
samples = [(x_test_noisy[i].squeeze() * 255).astype('uint8') for i in range(3)]

# Gradio интерфейс
interface = gr.Interface(
    fn=denoise,
    inputs=gr.Image(height=256, width=256, image_mode='L'),
    outputs=gr.Image(height=256, width=256, image_mode='L'),
    examples=samples
)
interface.launch()

Epoch 1/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - loss: 0.0421 - val_loss: 0.0153
Epoch 2/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.0146 - val_loss: 0.0137
Epoch 3/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0130 - val_loss: 0.0123
Epoch 4/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0120 - val_loss: 0.0117
Epoch 5/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0113 - val_loss: 0.0109
Epoch 6/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0108 - val_loss: 0.0105
Epoch 7/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0104 - val_loss: 0.0102
Epoch 8/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0101 - val_loss: 0.0100
Epoch 9/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0098 - val_loss: 0.0098
Epoch 10/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0097 - val_loss: 0.0097
Epoch 11/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0095 - val_loss: 0.0095
Epoch 12/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/ste

# Задание №.3 Реализуйте автоэнкодер для улучшения качества изображения путём увеличения его разрешения (апскейлинг-[определение](https://dic.academic.ru/dic.nsf/ruwiki/346555))

1. На входной слой нейронной сети подаётся изображение с размерностью (256, 256,3) - X_test. В результате работы нейронной сети на выходном слое должно получиться изображение (512, 512, 3) - Y_test. В итоге мы получаем двукратный апскейл исходного изображения.

2. Датасет собираете из изображений, разрешение которых выше эталонного (512,512,3). Затем преобразуете их в указанные размерности и формируете из них обучающую и проверочную выборку.

3. Обучите модель. Отобразите графики обучения

4. Подумайте, каким образом можно будет адаптировать модель вашей нейронной сети для двухратного апскейла изображения с любой размерностью, большей, чем (256,256,3), с полным или частичным сохранением его исходных пропорций. Например: подаём на вход изображение с разрешением (1920, 1080, 3) - получаем изображение с разрешением (3840, 2160, 3), т.е. с полным сохранением исходных пропорций или (1792, 1024, 3) с частичным сохранением исходных пропорций.

> Подсказка: Для этого можно реализовать алгоритм предварительной обработки исходного изображения, перед подачей его в нейронную сеть

In [ ]:
# Импорт библиотек
import tensorflow as tf
from tensorflow.keras import layers, Model
import gradio as gr
import cv2

# Загрузка и подготовка данных (пример с использованием случайных данных)
# В реальности используйте датасет с изображениями >512x512
# Например: https://www.tensorflow.org/datasets/catalog/div2k
# Для примера создадим искусственные данные
def load_data():
    (x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
    x_train = tf.image.resize(x_train, (256, 256))  # Имитация исходных данных
    x_train_low = tf.image.resize(x_train, (128, 128))
    return x_train_low / 255.0, x_train / 255.0

x_train_low, x_train_high = load_data()

# Архитектура автоэнкодера
input_img = layers.Input(shape=(128, 128, 3))

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Обучение
autoencoder.fit(
    x_train_low, x_train_high,
    epochs=20,
    batch_size=32,
    validation_split=0.2
)

# Функция для апскейлинга
def upscale(image):
    image = tf.image.resize(image, (128, 128))
    prediction = autoencoder.predict(np.expand_dims(image, axis=0))
    return tf.image.resize(prediction[0], (256, 256))

# Gradio интерфейс
samples = [x_train_low[i].numpy() for i in range(3)]
interface = gr.Interface(
    fn=upscale,
    inputs=gr.Image(shape=(128, 128)),
    outputs=gr.Image(shape=(256, 256)),
    examples=samples
)
interface.launch()